In [1]:
!conda install -c conda-forge --yes basemap-data-hires

Solving environment: ...working... done

# All requested packages already installed.



In [2]:
!conda install --yes pillow netCDF4

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\zsofi\Anaconda3

  added / updated specs: 
    - pillow


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    tk-8.6.7                   |       hcb92d03_3         3.5 MB
    certifi-2018.4.16          |           py36_0         143 KB
    kiwisolver-1.0.1           |   py36h12c3424_0          61 KB
    pillow-5.1.0               |   py36h0738816_0         648 KB
    freetype-2.8               |       h51f8f2c_1         468 KB
    libtiff-4.0.9              |       hb8ad9f9_1         831 KB
    matplotlib-2.2.2           |   py36h153e9ff_0         6.5 MB
    conda-4.5.4                |           py36_0         1.0 MB
    ------------------------------------------------------------
                                           Total:        13.1 MB

The following NEW packages will be INSTALLED:

    ki


tk-8.6.7             |  3.5 MB |            |   0% 
tk-8.6.7             |  3.5 MB | #2         |  12% 
tk-8.6.7             |  3.5 MB | ###6       |  37% 
tk-8.6.7             |  3.5 MB | ######1    |  61% 
tk-8.6.7             |  3.5 MB | #######6   |  76% 
tk-8.6.7             |  3.5 MB | ########6  |  86% 
tk-8.6.7             |  3.5 MB | #########3 |  94% 
tk-8.6.7             |  3.5 MB | #########9 | 100% 
tk-8.6.7             |  3.5 MB | ########## | 100% 

certifi-2018.4.16    |  143 KB |            |   0% 
certifi-2018.4.16    |  143 KB | 8          |   8% 
certifi-2018.4.16    |  143 KB | ########## | 100% 

kiwisolver-1.0.1     |   61 KB |            |   0% 
kiwisolver-1.0.1     |   61 KB | ########## | 100% 

pillow-5.1.0         |  648 KB |            |   0% 
pillow-5.1.0         |  648 KB | ########1  |  82% 
pillow-5.1.0         |  648 KB | ########## | 100% 

freetype-2.8         |  468 KB |            |   0% 
freetype-2.8         |  468 KB | ########5  |  85% 
freetyp

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

In [4]:
plt.figure(figsize=(8, 8))
m = Basemap(projection='ortho', resolution=None, lat_0=50, lon_0=-100)
m.bluemarble(scale=0.5);

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
C:\Users\zsofi\Anaconda3\lib\site-packages\mpl_toolkits\basemap\__init__.py:1711: MatplotlibDeprecationWarning: The axesPatch function was deprecated in version 2.1. Use Axes.patch instead.
  if limb is not ax.axesPatch:


In [ ]:
fig = plt.figure(figsize=(8, 8))
m = Basemap(projection='lcc', resolution=None,
            width=8E6, height=8E6, 
            lat_0=45, lon_0=-100,)
m.etopo(scale=0.5, alpha=0.5)

# Map (long, lat) to (x, y) for plotting
x, y = m(-122.3, 47.6)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, ' Seattle', fontsize=12);

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 8))

for i, res in enumerate(['l', 'h']):
    m = Basemap(projection='gnom', lat_0=57.3, lon_0=-6.2,
                width=90000, height=120000, resolution=res, ax=ax[i])
    m.fillcontinents(color="#FFDDCC", lake_color='#DDEEFF')
    m.drawmapboundary(fill_color="#DDEEFF")
    m.drawcoastlines()
    ax[i].set_title("resolution='{0}'".format(res));

In [ ]:
import pandas as pd
cities = pd.read_csv('california_cities.csv')

# Extract the data we're interested in
lat = cities['latd'].values
lon = cities['longd'].values
population = cities['population_total'].values
area = cities['area_total_km2'].values

In [ ]:
# 1. Draw the map background
fig = plt.figure(figsize=(8, 8))
m = Basemap(projection='lcc', resolution='h', 
            lat_0=37.5, lon_0=-119,
            width=1E6, height=1.2E6)
m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')

# 2. scatter city data, with color reflecting population
# and size reflecting area
m.scatter(lon, lat, latlon=True,
          c=np.log10(population), s=area,
          cmap='Reds', alpha=0.5)

# 3. create colorbar and legend
plt.colorbar(label=r'$\log_{10}({\rm population})$')
plt.clim(3, 7)

# make legend with dummy points
for a in [100, 300, 500]:
    plt.scatter([], [], c='k', alpha=0.5, s=a,
                label=str(a) + ' km$^2$')
plt.legend(scatterpoints=1, frameon=False,
           labelspacing=1, loc='lower left');

In [ ]:
from netCDF4 import Dataset
data = Dataset('gistemp250.nc')

from netCDF4 import date2index
from datetime import datetime
timeindex = date2index(datetime(2014, 1, 15),
                       data.variables['time'])

lat = data.variables['lat'][:]
lon = data.variables['lon'][:]
lon, lat = np.meshgrid(lon, lat)
temp_anomaly = data.variables['tempanomaly'][timeindex]

In [ ]:
fig = plt.figure(figsize=(10, 8))
m = Basemap(projection='lcc', resolution='c',
            width=8E6, height=8E6, 
            lat_0=45, lon_0=-100,)
m.shadedrelief(scale=0.5)
m.pcolormesh(lon, lat, temp_anomaly,
             latlon=True, cmap='RdBu_r')
plt.clim(-8, 8)
m.drawcoastlines(color='lightgray')

plt.title('January 2014 Temperature Anomaly')
plt.colorbar(label='temperature anomaly (°C)');